# MLP




## Import Dataset

Usually data is not ready

Using the MNIST dataset, I created a CNN model that recognizes handwritten digits with an accuracy of 99.64%. That is, it correctly predicted 4,972 out of 5,000 images.

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from datasets import load_dataset

# set print options to avoid scientific notation
torch.set_printoptions(sci_mode=False, precision=2)

# load the MNIST dataset
ds = load_dataset("ylecun/mnist")
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

Based on the dataset structure, we can access the images like this:

In [9]:
ds['train'][6]['image']

In [2]:
first_image = ds['train'][0]['image']
first_image

Please note that the images in the dataset are in PIL (Python Imaging Library) format.

In [ ]:
type(first_image)

## Data Preprocessing

Usually, the collected data is not immediately ready to be the input of a machine learning model. The steps taken to clean and prepare the data are known as **data preprocessing**.

When working with PIL images in deep learning, the typical workflow involves converting them first to NumPy arrays and then, to normalized PyTorch tensors. 

Suppose we have the following 4x4 PIL image:

```{figure} ../images/image-example.png
---
width: 180px
name: image-example
---
4x4 PIL image
```

When we convert this image to a NumPy array, each pixel value ranges from 0 (black) to 255 (white), with intermediate values representing varying shades of gray.

```python
np.array([[  0,  42,  85, 127],
          [ 42,  85, 127, 170],
          [ 85, 127, 170, 213],
          [127, 170, 213, 255]])
```

Neural networks perform better when when we **normalize** the input values so that they fall within the range [0, 1]. To do this, we divide each pixel value by 255, transforming the image into a format where 0 represents black and 1 represents white.

```python
tensor([[0.0000, 0.1647, 0.3333, 0.4980],
        [0.1647, 0.3333, 0.4980, 0.6667],
        [0.3333, 0.4980, 0.6667, 0.8353],
        [0.4980, 0.6667, 0.8353, 1.0000]])
```

We will now apply these transformations to all the images in the dataset.

In [ ]:
# convert PIL image to normalized PyToch tensor
def image_to_tensor(image):
    return torch.tensor(np.array(image)) / 255.0


def preprocess_data(split):
    
    x = []  # list to store image tensors
    y = []  # list to store labels

    for example in split:
        x.append(image_to_tensor(example['image']))
        y.append(example['label'])
    
    return torch.stack(x), torch.tensor(y)

train_x, train_y = preprocess_data(ds['train'])
test_x,  test_y  = preprocess_data(ds['test'])

## Train, Test, and Validation Splits

When working with neural networks, it is common practice to divide the dataset into three splits:

- The **train split** (80% of the dataset) is used to optimize the model parameters.
- The **validation split** (10% of the dataset) is used to optimize the hyperparameters, such as the size of the hidden layer, embedding size, and regularization strength.
- The **test split** (10% of the dataset) is used to evaluate the final performance of the model.

```{important}
The test split should be used only after the model is fully optimized, as its goal is to evaluate the model's performance on new data. Attempting to improve the test loss adjusting the hyperparameters of the model would result in overfitting.
```

<br>

(2.6)=
## Convolutional Neural Network

A **Convolutional Neural Network (CNN)** is a type of neural network primarly designed for tasks like image recognition and object detection.

CNNs outperform traditional neural netowrks when working with images becuase:
- They are more efficient, it is too much computation to train large-size images with different channels.
- They can capture the spatial dependencies of the image.
- They are not sensitive to the position of the object in the image.


CNNs works in two steps:
- **Feature extraction** is a phase where various filters and layers are applied to the images to extract information and features.
- **Classification** is a pahse where the image is classified based on the information extracted.


<br>

(2.7)=
## Convoultion Layer

The **convolution layer** extracts different features from the input image by applying different **filters** and producing several **feature maps**. 

```{figure} ../images/conv-layer-1.png
---
width: 340px
name: conv-layer-1
---
Convoultional Layer
```

To understand the math behind convolution layers, we will look at only one filter.

```{figure} ../images/conv-layer-2.png
---
width: 480px
name: conv-layer-2
---
Convoultional Layer 2
```

To calculate $m_{11}$, a fetaure map value, we will multiply the pixel values of the top-left 3 x 3 subregion of the input image with the filter values and add them up (sum product).

$$
m_{11} = i_{11}f_{11} + i_{12}f_{12} + i_{13}f_{13} + i_{21}f_{21} + i_{22}f_{22} + i_{23}f_{23} + i_{31}f_{31} + i_{32}f_{32} + i_{33}f_{33}
$$

To calculate $m_{12}$, we will select a 3 x 3 subregion one column to the right.

$$
m_{12} = i_{12}f_{11} + i_{13}f_{12} + i_{14}f_{13} + i_{22}f_{21} + i_{23}f_{22} + i_{24}f_{23} + i_{32}f_{31} + i_{33}f_{32} + i_{34}f_{33}
$$

The rest of the feature map values are calculated moving the 3x3 subregion similarlt. Thus, a convolution can be expressed as:
$$
m_{ij} = {f} \cdot {i}
$$

where:
- $i$ is the corresponfing subregion of the input
- $f$ is the filter

In [ ]:
# seed the random number generator for reproducibility
g = torch.Generator().manual_seed(1)

# intitialize input image randomly
image = torch.randn((6,6), generator=g)


n_filters = 5
filter_size = (3,3)

# intitialize filters randomly
filters = torch.randn((n_filters, filter_size[0], filter_size[1]), generator=g)

# dimensions of the feature maps
map_size = (image.shape[0] - filter_size[0] + 1, image.shape[1] - filter_size[1] + 1)

# initialize with zeros the feature maps
maps = torch.zeros((n_filters, map_size[0], map_size[1]))

# apply convolution
for i in range(map_size[0]):

    for j in range(map_size[1]):

        # extract the subregion of the input image
        subregion = image[i:i+filter_size[0], j:j+filter_size[1]]

        # update feature map at position (i,j) for all filters
        maps[:, i, j] = filters.view(-1, 9) @ subregion.reshape(-1)

# apply ReLU activation
maps = torch.relu(maps)

maps

<br>


## Pooling Layer

In [ ]:
pool_size = (2,2)       #  size of the pooling window

# dimensions of the feature maps after pooling
pooled_map_size = (maps.shape[1] // pool_size[0], maps.shape[2] // pool_size[1])

# initialize with zeros the pooled feature maps
pooled_maps = torch.zeros((n_filters, pooled_map_size[0], pooled_map_size[1]))

# apply max pooling
for i in range(pooled_map_size[0]):

    for j in range(pooled_map_size[1]):

        # Extract the 2x2 region for max pooling
        subregions = maps[:, 
                         i * pool_size[0]:(i+1) * pool_size[0], 
                         j * pool_size[1]:(j+1) * pool_size[1]]
        
        # Apply max pooling
        pooled_maps[:, i, j] = subregions.reshape(n_filters, -1).max(dim=1).values

pooled_maps

In [ ]:
n_filters = 30
filter_size = (5, 5)
pool_size = (2, 2)
n_hidden = 500      # number of neurons in hidden layers


# intitialize filter, weight matrices and bias vectors randomly
g = torch.Generator().manual_seed(1)

f1 = torch.randn((n_filters, filter_size[0], filter_size[1]), generator=g)

W1 = torch.randn((4320, n_hidden), generator=g)* 0.01
b1 = torch.randn(n_hidden, generator=g)

W2 = torch.randn((n_hidden, 10), generator=g)* 0.01
b2 = torch.randn(10, generator=g)

# parameters
parameters = [f1, W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True                     # track gradients

print(f"Number of parameters: {sum(p.nelement() for p in parameters)}")

In [ ]:
max_steps = 100   # train iterations
lr = 0.01         # learning rate
batch_size = 32   # number of examples per batch

track_loss = []

for step in range(max_steps):

    # batch construct
    ix = torch.randint(0, train_x.shape[0], (batch_size,))
    batch = train_x[ix]

    # apply convolution
    map_size = (batch.shape[1] - filter_size[0] + 1, batch.shape[2] - filter_size[1] + 1)
    maps = torch.zeros((batch_size, n_filters, map_size[0], map_size[1]))

    for i in range(map_size[0]):
        
        for j in range(map_size[1]):

            # extract the subregion for all images in the batch
            subregion = batch[:, i:i + filter_size[0], j:j + filter_size[1]]

            maps[:, :, i, j] = subregion.reshape(batch_size, -1) @ f1.view(-1, n_filters) # (32, 30, 24, 24) = (32, 25) @ (25, 30)
    
    maps = torch.relu(maps)
    
    # apply pool
    pooled_map_size = (maps.shape[2] // pool_size[0], maps.shape[3] // pool_size[1])
    pooled_maps = torch.zeros((batch_size, n_filters, pooled_map_size[0], pooled_map_size[1]))

    for i in range(pooled_map_size[0]):

        for j in range(pooled_map_size[1]):

            # extract the subregion for all images in the batch and filters
            subregion = maps[:, :, i * pool_size[0]:(i+1) * pool_size[0], j * pool_size[1]:(j+1) * pool_size[1]]

            pooled_maps[:, :, i, j] = subregion.reshape(batch_size, n_filters, -1).max(dim=2).values # (32, 30, 12, 12)
    
    # forward pass
    h1 = torch.relu(pooled_maps.view(batch_size, -1) @ W1 + b1) # (32, 100) = (32, 4320) x (4320, 100) + (100)
    h2 = torch.relu(h1 @ W2 + b2)                               # (32, 10) = (32, 100) x (100, 10) + (10)           

    # calculate loss
    loss = F.cross_entropy(h2, train_y[ix])
    track_loss.append(loss.item())
    if step % 3 == 0 or step == max_steps - 1:
        print(f"Step: {step:2d}/{max_steps}     Loss: {loss.item():.4f}")
        
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update parameters
    for p in parameters:
        p.data += -lr * p.grad

In [ ]:
-torch.tensor(1/10.0).log()

In [ ]:
print(f"After Convolution: {maps.mean().item()}, After ReLU: {maps.relu().mean().item()}")
print(f"After Hidden Layer 1: {h1.mean().item()}, After Hidden Layer 2: {h2.mean().item()}")


In [ ]:
plt.plot(track_loss[10:])
plt.xlabel('Step')
plt.ylabel('Loss');